In [3]:
import pandas as pd 
import nltk
import re
#Try also with encoding='ISO-8859-1' 
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin')


In [4]:
df.columns = ['pola', 'id', 'date', 'query', 'user', 'text']

In [5]:
# Remove URLs from tweets
df['clean_tweet'] = df['text'].apply(lambda x: re.sub(r'http\S+', '', x))

# Remove user mentions (@)
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(r'@[A-Za-z0-9_]+', '', x))

# remove numbers and special characters
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))

# to lowercase
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: x.lower())


In [ ]:
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# stop_words = stopwords.words('english')
# lemmatizer = WordNetLemmatizer()

# Tokenize tweets
# df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nltk.word_tokenize(x))

# Remove stop words and lemmatize tokens
#df['clean_tweet'] = df['clean_tweet'].apply(lambda tweet: [lemmatizer.lemmatize(word) for word in tweet if word not in stop_words])

# Join tokens back into a single string
#df['clean_tweet'] = df['clean_tweet'].apply(lambda tweet: ' '.join(tweet))

In [18]:
print(df['clean_tweet'].iloc[0])

is upset that he cant update his facebook by texting it and might cry as a result school today also blah


In [19]:
# Remove stop words and lemmatize tokens
#df['clean_tweet'] = df['clean_tweet'].apply(lambda tweet: [lemmatizer.lemmatize(word) for word in tweet if word not in stop_words])

# Join tokens back into a single string
#df['clean_tweet'] = df['clean_tweet'].apply(lambda tweet: ' '.join(tweet))

In [23]:
print(df['clean_tweet'].iloc[1])

i dived many times for the ball managed to save the rest go out of bounds


In [10]:
print(df['clean_tweet'].head())

0    is upset that he cant update his facebook by t...
1    i dived many times for the ball managed to sav...
2       my whole body feels itchy and like its on fire
3    no its not behaving at all im mad why am i her...
4                                   not the whole crew
Name: clean_tweet, dtype: object


In [14]:
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(r'\s+', ' ', x))


In [17]:
print(df['clean_tweet'].head(10))

0    is upset that he cant update his facebook by t...
1    i dived many times for the ball managed to sav...
2       my whole body feels itchy and like its on fire
3    no its not behaving at all im mad why am i her...
4                                   not the whole crew
5                                           need a hug
6    hey long time no see yes rains a bit only a bi...
7                              nope they didnt have it
8                                         que me muera
9               spring break in plain city its snowing
Name: clean_tweet, dtype: object


In [43]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   pola         1599999 non-null  int64 
 1   id           1599999 non-null  int64 
 2   date         1599999 non-null  object
 3   query        1599999 non-null  object
 4   user         1599999 non-null  object
 5   text         1599999 non-null  object
 6   clean_tweet  1599999 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB
None


In [44]:
print(df.columns)

Index(['pola', 'id', 'date', 'query', 'user', 'text', 'clean_tweet'], dtype='object')


In [24]:
# Drop unnecessary columns
df = df.drop(['id', 'date', 'query', 'user', 'text'], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   pola         1599999 non-null  int64 
 1   clean_tweet  1599999 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
None


In [25]:
df = df.dropna()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   pola         1599999 non-null  int64 
 1   clean_tweet  1599999 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
None


In [26]:

# Save cleaned dataset to a new CSV file
df.to_csv('preprocessed_tweets2.csv', index=False)